In [368]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization
import power_spectrum

### Add Directories/ Load Files

In [369]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [ ]:
fname = 'Derivative_sub006_3MFU_ses-2020102101410392_run-BrainSense20201021014600.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'test',
        fname
    ),
    info = None
)

subID = 'Sub005'

In [370]:
raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub029_FIF.fif'
    )
)

subID = 'Sub029'

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub029_FIF.fif...
Isotrak not found
    Range : 0 ... 80936 =      0.000 ...   323.744 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_16756\25762682.py:1: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub029_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join(


In [385]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [372]:
fft_name = str(subID)+'_FFT'
print(fft_name)

Sub029_FFT


In [ ]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

#plt.savefig(str(fft_fig)+str(fft_name),dpi = 300)

#np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [ ]:
#Saving the raw data as a fif file
raw2 = raw.copy()
Draw = raw2.get_data()
info = raw.info
new_raw = mne.io.RawArray(Draw, info)

#fif_name = os.path.join(project_path, 'data','Fifs','test/',str(subID)+'_FIF.fif')
#new_raw.save(fif_name)


In [384]:
#Zscore Normalization of raw data & plot of new figure
reload(baseline_correction)
data = Sxx[1,:,]
t = t
baseline = (None, None)
raw = raw
stim_ch = 0
bs_data = baseline_correction.baseline_corr(data, t, baseline, raw = raw, stim_ch = 5)

Applying baseline correction (mode: zscore)


In [ ]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_zscored',dpi = 300)
np.save(str(fft_file)+str(fft_name)+'_zscored'+'.npy',bs_data)

In [386]:
d = {'onset': [0, 208, 303],
    'duration': [30, 13, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

   onset  duration description
0      0        30     StimOff
1    208        13      StimOn
2    303        10     Rebound


In [387]:
## Print Power Spectra with Raw Data
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.4

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = str(subID)+ '_LSTN'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.5, title1)

plt.subplot(1,2,2)
title2 = str(subID)+ '_RSTN'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.2, title2)

plt.show()

In [388]:
## Printing the Power Spectra (RAW)
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

     StimOff_LSTN  StimOn_LSTN  Rebound_LSTN  StimOff_RSTN  StimOn_RSTN  \
0        0.447364     3.618580     46.245039      4.051099     6.232581   
1        4.396530    35.748907     95.880185     16.088837    48.344598   
2        4.640584    42.494694     23.208876     17.316446    57.070588   
3        2.826383    24.008829     15.461551     11.827067    44.355736   
4        2.524268    10.663635     11.027023      8.415378    17.175629   
..            ...          ...           ...           ...          ...   
121      0.000364     0.000290      0.000415      0.000325     0.000278   
122      0.000302     0.000272      0.000549      0.000389     0.000205   
123      0.000465     0.000398      0.000566      0.000389     0.000262   
124      0.000309     0.000402      0.000353      0.000270     0.000218   
125      0.000154     0.000153      0.000176      0.000120     0.000146   

     Rebound_RSTN  
0      120.788429  
1     1122.031954  
2      308.875397  
3      200.514792  

In [389]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','test/')
fft_file = os.path.join(project_path, 'data','power_spectra','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_PS',dpi = 150)
ps_df.to_csv(str(fft_file)+str(fft_name)+'_PS.csv')

In [390]:
## Print Power Spectra with Zscored Data
fig = plt.figure(figsize = (5,5))
ps_m1s0 = np.mean(bs_data[:,epoch_df.onset[0]:epoch_df.onset[0]+ epoch_df.duration[0]],1)
ps_m1s1 = np.mean(bs_data[:,epoch_df.onset[1]:epoch_df.onset[1]+ epoch_df.duration[1]],1)
ps_reb = np.mean(bs_data[:,epoch_df.onset[2]:epoch_df.onset[2]+ epoch_df.duration[2]],1)

plt.plot(np.arange(1,127),ps_m1s0,label = epoch_df.description[0])
plt.plot(np.arange(1,127),ps_m1s1,label = epoch_df.description[1])
plt.plot(np.arange(1,127),ps_reb,label = epoch_df.description[2])

plt.legend()

plt.xlim(60,90)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power (z-scored)')
plt.show()

ps_allzscored = np.transpose(np.squeeze(np.array([[ps_m1s0], [ps_m1s1],[ps_reb]])))
ps_allzscored_df = pd.DataFrame(ps_allzscored,
    columns = ['StimOff','StimOn','Rebound'],
    )
print(ps_allzscored_df)

      StimOff    StimOn   Rebound
0   -0.092693 -0.069759  1.134599
1   -0.121183 -0.061972  1.908965
2   -0.106008  0.086773  1.307858
3   -0.085334  0.175247  1.426205
4   -0.058365  0.028102  1.458706
..        ...       ...       ...
121  0.020875 -0.120096  0.655463
122  0.201924 -0.337744  0.271503
123  0.135961 -0.224558  0.014188
124 -0.024137 -0.168855  0.181751
125 -0.062146  0.060271 -0.140454

[126 rows x 3 columns]


In [391]:
plt.savefig(str(fft_fig)+str(fft_name)+'_zscoredPS',dpi = 150)
ps_allzscored_df.to_csv(str(fft_file)+str(fft_name)+'_zscoredPS.csv')

### Checking Spectral Width

In [392]:
#Import the power spectrum csv
ps_tbl = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub029_FFT_PS.csv'
))

subID = 'Sub029'

x1 = ps_tbl.StimOff_RSTN[50:101]
x2 = ps_tbl.StimOn_RSTN[50:101]
plt.plot(np.arange(0,51),x1)
plt.plot(np.arange(0,51),x2)
plt.xticks(np.arange(0,51,10), labels = np.arange(50,101,10))

plt.ylim(0,0.4)

(0.0, 0.4)

In [393]:
reload(power_spectrum)
%matplotlib qt

height = 0.04

peaks1, results_half1, peaks2, results_half2 = power_spectrum.powerSpectrum_width(x1, x2, height, subID)
print(results_half1)
print(results_half2)

[53 60 62 72 82]
[53 66 71]
[1.59607149 4.04580865 1.1557336  2.06562732 1.76301061]
[0.77663717 1.69021395 2.07905763]


In [394]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'Power_Spectra',
    'test',
    str(subID)+'_PSWidth'
),
dpi = 150)

### Average all Power Spectra and Plot them

In [ ]:
all_psSpontan = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSpontan_PsZscored.csv'
))


all_psSubharm = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSubharm_PsZscored.csv'
))

In [ ]:
all_psSpontan = all_psSpontan.drop('Sub029', axis=1)

In [ ]:
all_ps_mean1 = np.mean(all_psSpontan,1)
all_ps_sem1 = stats.sem(all_psSpontan,1)

all_ps_mean2 = np.mean(all_psSubharm,1)
all_ps_sem2 = stats.sem(all_psSubharm,1)

In [ ]:
plt.plot(np.arange(1,127), all_ps_mean1, label = 'Spontaneous FTG')
plt.fill_between(np.arange(1,127), all_ps_mean1-all_ps_sem1, all_ps_mean1+all_ps_sem1, alpha = 0.2)

plt.plot(np.arange(1,127), all_ps_mean2, label = 'DBS-Induced FTG')
plt.fill_between(np.arange(1,127), all_ps_mean2-all_ps_sem2, all_ps_mean2+all_ps_sem2, alpha = 0.2)

plt.xlim(50, 100)
plt.ylim(-0.4, 2)
plt.legend()

plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power [z-scored]')
plt.title('Averaged Power Spectra [N = 6]')

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'Avg_PS'
),
dpi = 200)

In [400]:
###Compare Power Spectra Widths

widths_tbl = pd.read_csv(os.path.join(
    project_path,
    'results',
    'PS_Widths.csv'
))
print(widths_tbl)

    SubID  PeakMed  PeakSuh  WidthMed  WidthSubh
0  Sub005       77       65  2.958513   1.613145
1  Sub006       82       65  2.254888   1.256823
2  Sub009       73       65  2.066663   1.257691
3  Sub014       74       65  2.184958   1.324285
4  Sub017       76       63  2.317574   2.179645
5  Sub029       82       71  1.763011   2.079058


In [399]:
p_value = mlxtend.permutation_test(
    widths_tbl['WidthMed'], widths_tbl['WidthSubh'], paired=True, method="approximate", seed=0, num_rounds=100000
)

print(p_value)

0.06327936720632793
